In [2]:
from projecttracker.utils import file_handler

In [1]:
import os
os.chdir('/Users/prana/Documents/MDS/Project-Group-13')

In [15]:


class Operations:
    def __init__(self):
        self.projects = {}
        self.tasks = {}
        
    #def view
        
    def add_proj(self, **kwargs):
        new_proj = Project(**kwargs)
        self.projects[new_proj.projectID] = new_proj
        print(f"Project '{new_proj.projectID}' added.")
        file_handler.write_to_json(new_proj, 'project.json')  # Write to JSON file
        return new_proj

    def add_task(self, **kwargs):
        new_task = Task(**kwargs)
        self.tasks[new_task.taskID] = new_task
        print(f"Task '{new_task.taskID}' added.")
        file_handler.write_to_json(new_task, 'task.json')  # Write to JSON file
        return new_task
    
    def modify_item(self):
        
        modify_type = input("Enter 'project' or 'task' ID to modify: ").upper()
        
        while True:

            if modify_type[0] == 'P' and modify_type in self.projects:
                project = self.projects[modify_type]
                print(f"Current attributes of Project '{modify_type}':")
                for key, value in project.__dict__.items():
                     print(f"{key}: {value}")

                attribute = input("Enter the attribute to modify: ")
                if hasattr(project, attribute):
                    new_value = input(f"Enter new value for '{attribute}': ")
                    setattr(project, attribute, new_value)
                    print(f"Attribute '{attribute}' updated for Project '{modify_type}'.")
                else:
                    print(f"Attribute '{attribute}' does not exist in the project.")
            elif modify_type[0] == 'T' and modify_type in self.tasks:
                task = self.tasks[modify_type]
                print(f"Current attributes of Task '{modify_type}':")
                for key, value in task.__dict__.items():
                    print(f"{key}: {value}")

                attribute = input("Enter the attribute to modify: ")
                if hasattr(task, attribute):
                    new_value = input(f"Enter new value for '{attribute}': ")
                    setattr(task, attribute, new_value)
                    print(f"Attribute '{attribute}' updated for Task '{modify_type}'.")
                else:
                    print(f"Attribute '{attribute}' does not exist in the task.")
            else:
                print("Item not found or invalid item type.")
            
            if input(f"Do you want to continue updating {modify_type} (Y/N)?") == 'N':
                break
    
    def delete_item(self):
        delete_type = input("Enter 'project' or 'task' ID to delete: ").upper()
        
        if delete_type[0] == 'P':
            if delete_type in self.projects:
                del_proj = self.projects.pop(delete_type)
                print(f"Project '{delete_type}' deleted.")
                return del_proj
            else:
                print(f"Project '{delete_type}' not found.")
                return None
        elif delete_type[0] == 'T':
            if delete_type in self.tasks:
                del_task = self.tasks.pop(delete_type)
                print(f"Task '{delete_type}' deleted.")
                return del_task
            else:
                print(f"Task '{delete_type}' not found.")
                return None
        else:
            print("Invalid input. Please enter 'project' or 'task'.")
            return None
    

class Project(Operations):
    #project_counter = 0
    def __init__(self, Name, Priority, Duration, Comments, assignedTo, startDate, Deadline, Owner):
        project_id = self.get_next_project_id()
        self.projectID = f'P{project_id:04}'
        self.projectName = Name
        self.projectPriority = Priority
        self.projectDuration = Duration
        self.projectComments = Comments
        self.assignedToProjectTL = assignedTo
        self.projectStartDate = startDate 
        self.projectDeadline = Deadline
        self.projectOwner = Owner
        self.IsProjectCompleted = 'N'
    
    def get_next_project_id(self):
        last_project_id = 0
        try:
            projects_from_json = file_handler.read_from_json('project.json')
        except:
            pass
        else:
            project_ids = [project['projectID'] for project in projects_from_json]
            last_inserted_project_id = max(project_ids, key=lambda x: int(x[1:]))
            last_project_id = int(last_inserted_project_id[1:])
        finally:
            last_project_id += 1
            return last_project_id
    


class Task(Project):
    #task_counter = 0
    def __init__(self, projectID, Name, Priority, Duration, Comments, assignedTo, startDate, Deadline):
        self.projectID = projectID
        task_id = self.get_next_task_id()
        self.taskID = f'T{task_id:04}'
        self.taskName = Name
        self.taskPriority = Priority
        self.taskDuration = Duration
        self.taskComments = Comments
        self.assignedToTask = assignedTo
        self.taskStartDate = startDate
        self.taskDeadline = Deadline
        self.IsTaskCompleted = 'N'
    
    def get_next_task_id(self):
        last_task_id = 0
        try:
            tasks_from_json = file_handler.read_from_json('task.json')
        except:
            pass
        else:
            task_ids = [task['taskID'] for task in tasks_from_json]
            last_inserted_task_id = max(task_ids, key=lambda x: int(x[1:]))
            last_task_id = int(last_inserted_task_id[1:])
        finally:
            last_task_id += 1
            return last_task_id
    
    



In [17]:
operations = Operations()

project_data = {
    'Name': 'Project 1',
    'Priority': 'High',
    'Duration': '2 weeks',
    'Comments': 'Important project',
    'assignedTo': 'Team Lead 1',
    'startDate': '2023-01-01',
    'Deadline': '2023-01-15',
    'Owner': 'Project Owner 1',
}
operations.add_proj(**project_data)

# Add a task
task_data = {
    'projectID': 'P0001',
    'Name': 'Task 1',
    'Priority': 'Medium',
    'Duration': '1 week',
    'Comments': 'Urgent task',
    'assignedTo': 'Team Member 1',
    'startDate': '2023-01-02',
    'Deadline': '2023-01-08',
}
operations.add_task(**task_data)

# Modify a project or task
#operations.modify_item()

# Delete a project or task
#operations.delete_item()

# Read projects and tasks from JSON files
projects_from_json = file_handler.read_from_json('project.json')
tasks_from_json = file_handler.read_from_json('task.json')
#tasks_from_json = read_from_json()

# Display the read projects and tasks
print("\nProjects read from JSON:")
for project_data in projects_from_json:
    print(project_data)

print("\nTasks read from JSON:")
for task_data in tasks_from_json:
    print(task_data)

Project 'P0002' added.
Task 'T0002' added.

Projects read from JSON:
{'projectID': 'P0001', 'projectName': 'Project 1', 'projectPriority': 'High', 'projectDuration': '2 weeks', 'projectComments': 'Important project', 'assignedToProjectTL': 'Team Lead 1', 'projectStartDate': '2023-01-01', 'projectDeadline': '2023-01-15', 'projectOwner': 'Project Owner 1', 'IsProjectCompleted': 'N'}
{'projectID': 'P0002', 'projectName': 'Project 1', 'projectPriority': 'High', 'projectDuration': '2 weeks', 'projectComments': 'Important project', 'assignedToProjectTL': 'Team Lead 1', 'projectStartDate': '2023-01-01', 'projectDeadline': '2023-01-15', 'projectOwner': 'Project Owner 1', 'IsProjectCompleted': 'N'}

Tasks read from JSON:
{'projectID': 'P0001', 'taskID': 'T0001', 'taskName': 'Task 1', 'taskPriority': 'Medium', 'taskDuration': '1 week', 'taskComments': 'Urgent task', 'assignedToTask': 'Team Member 1', 'taskStartDate': '2023-01-02', 'taskDeadline': '2023-01-08', 'IsTaskCompleted': 'N'}
{'projectID

In [18]:
tasks_from_json

[{'projectID': 'P0001',
  'taskID': 'T0001',
  'taskName': 'Task 1',
  'taskPriority': 'Medium',
  'taskDuration': '1 week',
  'taskComments': 'Urgent task',
  'assignedToTask': 'Team Member 1',
  'taskStartDate': '2023-01-02',
  'taskDeadline': '2023-01-08',
  'IsTaskCompleted': 'N'},
 {'projectID': 'P0001',
  'taskID': 'T0002',
  'taskName': 'Task 1',
  'taskPriority': 'Medium',
  'taskDuration': '1 week',
  'taskComments': 'Urgent task',
  'assignedToTask': 'Team Member 1',
  'taskStartDate': '2023-01-02',
  'taskDeadline': '2023-01-08',
  'IsTaskCompleted': 'N'}]

In [19]:
projects_from_json

[{'projectID': 'P0001',
  'projectName': 'Project 1',
  'projectPriority': 'High',
  'projectDuration': '2 weeks',
  'projectComments': 'Important project',
  'assignedToProjectTL': 'Team Lead 1',
  'projectStartDate': '2023-01-01',
  'projectDeadline': '2023-01-15',
  'projectOwner': 'Project Owner 1',
  'IsProjectCompleted': 'N'},
 {'projectID': 'P0002',
  'projectName': 'Project 1',
  'projectPriority': 'High',
  'projectDuration': '2 weeks',
  'projectComments': 'Important project',
  'assignedToProjectTL': 'Team Lead 1',
  'projectStartDate': '2023-01-01',
  'projectDeadline': '2023-01-15',
  'projectOwner': 'Project Owner 1',
  'IsProjectCompleted': 'N'}]

In [ ]:
project_ids = [project['projectID'] for project in projects_from_json]

    # Find the last inserted projectID
last_inserted_project_id = max(project_ids, key=lambda x: int(x[1:]))

In [ ]:
int(last_inserted_project_id[1:])

In [22]:
import pandas as pd
# Convert data to pandas DataFrame
task_df = pd.DataFrame(tasks_from_json)
project_df = pd.DataFrame(projects_from_json)

# Merge the two DataFrames on 'projectID'
result_df = pd.merge(project_df, task_df, on='projectID', how='left')

# Display the result
result_df.head()

,projectID,projectName,projectPriority,projectDuration,projectComments,assignedToProjectTL,projectStartDate,projectDeadline,projectOwner,IsProjectCompleted,taskID,taskName,taskPriority,taskDuration,taskComments,assignedToTask,taskStartDate,taskDeadline,IsTaskCompleted
0,P0001,Project 1,High,2 weeks,Important project,Team Lead 1,2023-01-01,2023-01-15,Project Owner 1,N,T0001,Task 1,Medium,1 week,Urgent task,Team Member 1,2023-01-02,2023-01-08,N
1,P0001,Project 1,High,2 weeks,Important project,Team Lead 1,2023-01-01,2023-01-15,Project Owner 1,N,T0002,Task 1,Medium,1 week,Urgent task,Team Member 1,2023-01-02,2023-01-08,N
2,P0002,Project 1,High,2 weeks,Important project,Team Lead 1,2023-01-01,2023-01-15,Project Owner 1,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
